In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import os
import time

### Links:

In [ ]:
# Evergore Log In Page
evergore_log_in = 'https://www.evergore.de/login'

# in DevTools, look for the value of the character-choice button
character_choice = "button[value='xxxxxxxxxxx']" 

# Group-Report-Message with the Battle-Report Links
gruppenbericht_message = "xxxxxxxxxxx"

# Username here
user_name = "xxxxxxxxxxx"

### Set Up driver and connect to Website:

In [3]:
# Set up the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Step 1: Open the website
driver.get(evergore_log_in)

### Create Secret.txt for password:

In [ ]:
# create secret file for password
secrets_file = open("secrets.txt","r")
pass_word = secrets_file.read()

### Enter Log In Credentials:

In [ ]:
# Log in
# Locate username and password fields and enter credentials
username = driver.find_element(By.ID, "nameInput")  # Use the correct selector
password = driver.find_element(By.ID, "pwInput")  # Use the correct selector

username.send_keys(user_name)  
password.send_keys(pass_word) 

# Submit the form (use appropriate selector, could be a button or form submission)
password.send_keys(Keys.RETURN)

TypeError: 'WebElement' object is not iterable

### Hit Log in Button:

In [ ]:
# Log In
login_button = driver.find_element(By.CSS_SELECTOR, character_choice)
login_button.click()

### Navigate to GroupReport and scrape all Links:

In [ ]:
# Navigate to the sub-page (if necessary)
driver.get(gruppenbericht_message)

# Scrape all links on the sub-page
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "a")))

# Find all <a> elements and get their href attribute
links = driver.find_elements(By.TAG_NAME, "a")
urls = [link.get_attribute('href') for link in links if link.get_attribute('href') is not None]

# Print the links (or save to file, etc.)
# for url in urls:
#     print(url)

### Extract BattleReport Links:

In [ ]:
# extract "Kampfberichte"-Links
kampfberichte = [url for url in urls if "battle_report&battle_id" in url]
# kampfberichte

### Determine File Name (Unique IDs):

In [ ]:
# create Directory for files, if not already existing
output_directory = "Kampfberichte"
os.makedirs(output_directory, exist_ok=True)

# check files
def get_last_file_number(directory, base_name, extension=".html"):
    existing_files = os.listdir(directory)
    last_number = 0
    
# find highest number
    for filename in existing_files:
        if filename.startswith(base_name) and filename.endswith(extension):

            try:
                number = int(filename[len(base_name):-len(extension)].strip("_"))
                if number > last_number:
                    last_number = number
            except ValueError:
                continue

    return last_number

# give new file highest counter
base_file_name = "Kampfbericht"
last_number = get_last_file_number(output_directory, base_file_name)
next_number = last_number + 1  # Fortlaufende Nummer
file_name = f"{base_file_name}_{next_number}.html"
file_path = os.path.join(output_directory, file_name)

### Scrape BattleReports and Save into 1 File:

In [ ]:
# Create Document for BattleReports
with open(file_path, 'w', encoding='utf-8') as file:


    # iterate link-list
    for url in kampfberichte:
        
        # open links
        driver.get(url)

        # wait 5 sec.
        driver.implicitly_wait(5) 

        # Extract HTML- Code
        page_content = driver.page_source

        # write into File
        file.write(page_content)
        
        # be nice, sleep for a sec
        time.sleep(2)

# close WebDriver
driver.quit()

print("Alle Seiten wurden erfolgreich gespeichert!")